전에 과제로 수행했었던 파이썬 코드입니다.

밑의 코드들은 제주도 내 버스정류장들의 날씨를 측정하기 위해
지리적으로 정류장이 밀집되어 있는 기상 측정소인 '제주 측정소', '고산 측정소', '성산 측정소', '서귀포 측정소'를 임의로 설정하고, 그곳의 날씨를 가까운 정류장들에 적용하는 과정입니다.

일단 날씨 정보는 제외하고 지도와 마커 생성, 그리고 특정 측정소에서 가까운 정류장들을 구하는 코드들만 정리하였습니다.

저의 구글 드라이브에 마운트하였기 때문에 실행되지는 않을 것입니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd #데이터프레임
import numpy as np #행렬 생성
from collections import Counter #컨테이너형 데이터
import geopy.distance #거리 계산
import folium #지도 라이브러리
from folium.plugins import MarkerCluster #지리 데이터 가공

In [ ]:
train = pd.read_csv('/content/drive/My Drive/python/train.csv') #csv 파일로 데이터프레임 생성

In [ ]:
train.head() #데이터 형태 확인

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,6~7_ride,7~8_ride,8~9_ride,9~10_ride,10~11_ride,11~12_ride,6~7_takeoff,7~8_takeoff,8~9_takeoff,9~10_takeoff,10~11_takeoff,11~12_takeoff,18~20_ride
0,0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.49373,0.0,1.0,2.0,5.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.48508,1.0,4.0,4.0,2.0,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.47352,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,3,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.49252,0.0,17.0,6.0,26.0,14.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0
4,4,2019-09-01,4270000,시내,1646,중문관광단지입구,33.25579,126.41260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
jeju=(33.51411, 126.52969) 
gosan=(33.29382, 126.16283) 
seongsan=(33.38677, 126.8802)
po=(33.24616, 126.5653) #각 측정소의 위도와 경도 설정
map_osm1= folium.Map((33.399835, 126.506031),zoom_start=9, tiles='Stamen Toner') #해당 좌표의 map 생성 #Stamen Toner: 도로 강조

In [ ]:
mc = MarkerCluster() #마커 생성

mc.add_child(folium.Marker(location=jeju,popup='제주 측정소',icon=folium.Icon(color='red',icon='info-sign'))) #각 Marker 정보를 add_child를 이용해서 입력
map_osm1.add_child(mc) #MarkerCluster를 basemap에 add_child를 통해서 입력

mc.add_child(folium.Marker(location=gosan,popup='고산 측정소',icon=folium.Icon(color='red',icon='info-sign')))
map_osm1.add_child(mc) 

mc.add_child(folium.Marker(location=seongsan,popup='성산 측정소',icon=folium.Icon(color='red',icon='info-sign')))
map_osm1.add_child(mc) 

mc.add_child(folium.Marker(location=po,popup='서귀포 측정소',icon=folium.Icon(color='red',icon='info-sign')))
map_osm1.add_child(mc)

In [ ]:
data=train[['latitude','longitude','station_name']].drop_duplicates(keep='first')
#train 데이터의 위 열을 가지고 새로운 데이터프레임(data) 생성, 중복값을 제거하되 중복된 값 중 첫번째 값을 남김

data2=data.groupby(['station_name'])['latitude','longitude'].mean() 
#data에서 'station'으로 group by 후 각 위도와 경도의 평균을 값으로 한 데이터프레임(data2) 생성

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.


In [ ]:
data2.to_csv("folium.csv")
data2=pd.read_csv("folium.csv")
#data2를 csv 파일로 저장 후 데이터프레임 형태로 불러옴(저장된 파일은 왼쪽 파일 탭에서 확인 가능)

In [ ]:
map_osm2= folium.Map((33.399835, 126.506031),zoom_start=9)
for row in data2.itertuples(): #튜플 형태의 data2에서 row를 반복
    mc.add_child(folium.Marker(location=[row.latitude, row.longitude], popup=row.station_name)) 
    #row.latitude, row.longtitude를 받아 마커 위치를 생성하고, row.station_name을 받아 popup 생성
    map_osm2.add_child(mc) #map_osm2 지도에 마커 찍어줌

In [ ]:
map_osm2 #지도는 확대/축소 가능

In [ ]:
q1 = [geopy.distance.geodesic( (i,j), jeju).km for i,j in list( zip( train['latitude'],train['longitude'] )) ]
q2 = [geopy.distance.geodesic( (i,j), gosan).km for i,j in list( zip( train['latitude'],train['longitude'] )) ]
q3 = [geopy.distance.geodesic( (i,j), seongsan).km for i,j in list( zip( train['latitude'],train['longitude'] )) ]
q4 = [geopy.distance.geodesic( (i,j), po).km for i,j in list( zip( train['latitude'],train['longitude'] )) ]
#for~zip: zip() 안의 반환값들을 for을 이용해 루프를 돌면서 튜플값 차례로 출력
#geopy.distance.geodesic(a, b, c, d).km: km 단위로 (a,b)와 (c,d) 사이의 거리 구하기

train['dis_jeju']=q1
train['dis_gosan']=q2
train['dis_seongsan']=q3
train['dis_po']=q4

In [ ]:
total=pd.DataFrame(list(zip( q1,q2,q3,q4)),columns=['jeju','gosan','seongsan','po'])
#pd.DAataFrame(데이터,지정할 colunms): 데이터프레임 형성

In [ ]:
train['dist_name'] = total.apply(lambda x: x.argmin(), axis=1)
#lambda 매개변수:표현식
#axis=1: 행 방향으로, argmin: 행렬에서 최소값 색인(column 위치를 반환)

In [ ]:
train['dist_name'] = train['dist_name'].map({0:'jeju',
                                            1:'gosan',
                                            2:'seongsan',
                                            3:'po'}) #색인값을 column명으로 변환

In [ ]:
data3=train[['station_name','latitude','longitude','dist_name']].drop_duplicates(keep='first')
#위 컬럼을 갖는 데이터프레임 생성(data3)

In [ ]:
Counter(data3['dist_name'])
#컬럼 내에서 특정 데이터 값을 몇 개씩 가지고 있는지(각 측정소에서 가장 가까운 정류장이 몇 개인지 확인 가능)

Counter({'gosan': 659, 'jeju': 1479, 'po': 828, 'seongsan': 597})